# Recommender Systems Tutorial
In this tutorial, we'll provide a simple walkthrough of how to use Snorkel to build a recommender system.
We consider a setting similar to the [Netflix challenge](https://www.kaggle.com/netflix-inc/netflix-prize-data), but with books instead of movies.
We have a set of users and books, and for each user we know the set of books they have interacted with (read or marked as to-read).
We don't have the user's numerical ratings for the books they read, except in a small number of cases.
We also have some text reviews written by users.

Our goal is to build a recommender system by training a classifier to predict whether a user will read and like any given book.
We represent users using the set of books they have interacted with, and train a model over a user-book pair to predict a `rating` (a `rating` of 1 means the user will read and like the book).
We represent a user by the set of books they interacted with.
Once we have this model trained, we can use it to recommend books to a user when they visit the site.
For example, we can just predict the rating for the user paired with a book for a few thousand likely books, then pick the books with the top ten predicted ratings.

Of course, there are many other ways to approach this problem.
The field of [recommender systems](https://en.wikipedia.org/wiki/Recommender_system) is a very well studied area with a wide variety of settings and approaches, and we just focus on one of them.

We will use the [Goodreads](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home) dataset, from
"Item Recommendation on Monotonic Behavior Chains", RecSys'18 (Mengting Wan, Julian McAuley), and "Fine-Grained Spoiler Detection from Large-Scale Review Corpora", ACL'19 (Mengting Wan, Rishabh Misra, Ndapa Nakashole, Julian McAuley).
In this dataset, we have user interactions and reviews for Young Adult novels from the Goodreads website, along with metadata (like `title` and `authors`) for the novels.

In [1]:
import os

if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("recsys")

## Loading Data

We start by running the `download_and_process_data` function.
The function returns the `df_train`, `df_test`, `df_dev`, `df_val` dataframes, which correspond to our training, test, development, and validation sets.
Each of those dataframes has the following fields:
* `user_idx`: A unique identifier for a user.
* `book_idx`: A unique identifier for a book that is being rated by the user.
* `book_idxs`: The set of books that the user has interacted with (read or planned to read).
* `review_text`: Optional text review written by the user for the book.
* `rating`: Either `0` (which means the user did not read or did not like the book) or `1` (which means the user read and liked the book). The `rating` field is missing for `df_train`.
Our objective is to predict whether a given user (represented by the set of book_idxs the user has interacted with) will read and like any given book.
That is, we want to train a model that takes a set of `book_idxs` (the user) and a single `book_idx` (the book to rate) and predicts the `rating`.

In addition, `download_and_process_data` also returns the `df_books` dataframe, which contains one row per book, along with metadata for that book (such as `title` and `first_author`).

In [2]:
from utils import download_and_process_data

(df_train, df_test, df_dev, df_val), df_books = download_and_process_data()

df_books.head()

/home/ubuntu/snorkel-tutorials/recsys/utils.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_interactions_nz["rating_4_5"] = df_interactions_nz.rating.map(ratings_map)


,authors,average_rating,book_id,country_code,description,is_ebook,language_code,ratings_count,similar_books,text_reviews_count,title,first_author,book_idx
3,[293603],4.35,10099492,US,It all comes down to this.\nVlad's running out...,True,eng,152,"[25861113, 7430195, 18765937, 6120544, 3247550...",9,Twelfth Grade Kills (The Chronicles of Vladimi...,293603,0
4,[4018722],3.71,22642971,US,The future world is at peace.\nElla Shepherd h...,True,eng,1525,"[20499652, 17934493, 13518102, 16210411, 17149...",428,The Body Electric,4018722,1
5,[6537142],3.89,31556136,US,A gorgeously written and deeply felt literary ...,True,,109,[],45,Like Water,6537142,2
12,"[6455200, 5227552]",3.90,18522274,US,Zoe Vanderveen is on the run with her captor t...,True,en-US,191,"[25063023, 18553080, 17567752, 18126509, 17997...",6,"Volition (The Perception Trilogy, #2)",6455200,3
13,[187837],3.19,17262776,US,"The war is over, but for thirteen-year-old Rac...",True,eng,248,"[16153997, 10836616, 17262238, 16074827, 13628...",68,Little Red Lies,187837,4


We look at a sample of the labeled development set.
As an example, we want our final recommendations model to be able to predict that a user who has interacted with `book_idxs` (25743, 22318, 7662, 6857, 83, 14495, 30664, ...) would either not read or not like the book with `book_idx` 22764 (first row), while a user who has interacted with `book_idxs` (3880, 18078, 9092, 29933, 1511, 8560, ...) would read and like the book with `book_idx` 3181 (second row).

In [3]:
df_dev.sample(frac=1, random_state=12).head()

,user_idx,book_idxs,book_idx,rating,review_text
32280,1260,"(17578, 30070, 20410, 5680, 22365, 27003, 1956...",14953,1,NaN
242706,9388,"(8436, 16671, 31625, 31132, 825, 346, 13186, 5...",31566,0,Review from my book blog (wordpress / livejour...
739371,28757,"(6775, 19163, 4093, 6444, 21131, 6885, 4404, 2...",17127,0,The first half was boring. It took over half t...
92040,3591,"(17123, 4234, 27418, 24762, 1529, 4905, 1753, ...",11381,1,NaN
683153,26546,"(2890, 17625, 16365, 3088, 19572, 25743, 16030...",6705,1,NaN


## Writing Labeling Functions

In [4]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

If a user has interacted with several books written by an author, there is a good chance that the user will read and like other books by the same author.
We express this as a labeling function, using the `first_author` field in the `df_books` dataframe.
We picked the threshold 15 by plotting histograms and running error analysis using the dev set.

In [5]:
from snorkel.labeling.lf import labeling_function

book_to_first_author = dict(zip(df_books.book_idx, df_books.first_author))
first_author_to_books_df = df_books.groupby("first_author")[["book_idx"]].agg(set)
first_author_to_books = dict(
    zip(first_author_to_books_df.index, first_author_to_books_df.book_idx)
)


@labeling_function(resources=dict(book_to_first_author=book_to_first_author, first_author_to_books=first_author_to_books))
def shared_first_author(x, book_to_first_author, first_author_to_books):
    author = book_to_first_author[x.book_idx]
    same_author_books = first_author_to_books[author]
    num_read = len(set(x.book_idxs).intersection(same_author_books))
    return POSITIVE if num_read > 15 else ABSTAIN

We can also leverage the long text reviews written by users to guess whether they liked or disliked a book.
For example, the third `df_dev` entry above has a review with the text `'4.5 STARS'`, which indicates that the user liked the book.
We write a simple LF that looks for similar phrases to guess the user's rating of a book.
We interpret >= 4 stars to indicate a positive rating, while < 4 stars is negative.

In [6]:
low_rating_strs = [
    "one star",
    "1 star",
    "two star",
    "2 star",
    "3 star",
    "three star",
    "3.5 star",
    "2.5 star",
    "1 out of 5",
    "2 out of 5",
    "3 out of 5",
]
high_rating_strs = ["5 stars", "five stars", "four stars", "4 stars", "4.5 stars"]


@labeling_function(resources=dict(low_rating_strs=low_rating_strs, high_rating_strs=high_rating_strs))
def stars_in_review(x, low_rating_strs, high_rating_strs):
    if not isinstance(x.review_text, str):
        return ABSTAIN
    for low_rating_str in low_rating_strs:
        if low_rating_str in x.review_text.lower():
            return NEGATIVE
    for high_rating_str in high_rating_strs:
        if high_rating_str in x.review_text.lower():
            return POSITIVE
    return ABSTAIN

We can also run [TextBlob](https://textblob.readthedocs.io/en/dev/index.html), a tool that provides a pretrained sentiment analyzer, on the reviews, and use its polarity and subjectivity scores to estimate the user's rating for the book.
As usual, these thresholds were picked by analyzing the score distributions and running error analysis.

In [7]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob


@preprocessor(memoize=True)
def textblob_polarity(x):
    if isinstance(x.review_text, str):
        x.blob = TextBlob(str(x.review_text))
    else:
        x.blob = None
    return x


# Label high polarity reviews as positive.
@labeling_function(pre=[textblob_polarity])
def polarity_positive(x):
    if x.blob:
        if x.blob.polarity > 0.3:
            return POSITIVE
    return ABSTAIN


# Label high subjectivity reviews as positive.
@labeling_function(pre=[textblob_polarity])
def subjectivity_positive(x):
    if x.blob:
        if x.blob.subjectivity > 0.75:
            return POSITIVE
    return ABSTAIN


# Label low polarity reviews as negative.
@labeling_function(pre=[textblob_polarity])
def polarity_negative(x):
    if x.blob:
        if x.blob.polarity < 0.0:
            return NEGATIVE
    return ABSTAIN

In [8]:
from snorkel.labeling import PandasLFApplier, LFAnalysis

lfs = [
    stars_in_review,
    shared_first_author,
    polarity_positive,
    subjectivity_positive,
    polarity_negative,
]

applier = PandasLFApplier(lfs)
L_dev = applier.apply(df_dev)
LFAnalysis(L_dev, lfs).lf_summary(df_dev.rating)

  0%|          | 0/83287 [00:00<?, ?it/s]

  0%|          | 51/83287 [00:00<02:43, 508.75it/s]

  0%|          | 158/83287 [00:00<02:51, 485.21it/s]

  0%|          | 301/83287 [00:00<02:17, 604.91it/s]

  1%|          | 433/83287 [00:00<01:54, 721.90it/s]

...

100%|█████████▉| 82949/83287 [01:16<00:00, 1144.10it/s]

100%|█████████▉| 83081/83287 [01:16<00:00, 1185.33it/s]

100%|█████████▉| 83219/83287 [01:16<00:00, 1236.57it/s]

100%|██████████| 83287/83287 [01:16<00:00, 1081.95it/s]

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
stars_in_review,0,"[0, 1]",0.016449,0.004298,0.001417,1126,244,0.821898
shared_first_author,1,[1],0.052037,0.001429,0.000360,3212,1122,0.741117
polarity_positive,2,[1],0.048315,0.013952,0.000744,3263,761,0.810885
subjectivity_positive,3,[1],0.018838,0.013928,0.002846,1209,360,0.770554
polarity_negative,4,[0],0.016821,0.004106,0.003374,760,641,0.542470


### Applying labeling functions to the training set

We apply the labeling functions to the training set, and then filter out examples unlabeled by any LF, and combine the rest with the dev set to form our final training set.
We choose to train on our dev set in this case because it's relatively large (85.3k examples) and we didn't overfit to it when writing our LFs.

In [9]:
from snorkel.labeling.model.label_model import LabelModel

L_train = applier.apply(df_train)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=5000, seed=123, log_freq=20, lr=0.01)
Y_train_preds = label_model.predict(L_train)

  0%|          | 0/599105 [00:00<?, ?it/s]

  0%|          | 13/599105 [00:00<1:16:53, 129.85it/s]

  0%|          | 159/599105 [00:00<55:52, 178.67it/s] 

  0%|          | 252/599105 [00:00<42:20, 235.74it/s]

  0%|          | 412/599105 [00:00<31:31, 316.58it/s]

...

100%|█████████▉| 598623/599105 [09:17<00:00, 1260.76it/s]

100%|█████████▉| 598756/599105 [09:17<00:00, 1177.15it/s]

100%|█████████▉| 598880/599105 [09:18<00:00, 1112.32it/s]

100%|█████████▉| 598996/599105 [09:18<00:00, 742.31it/s] 

100%|██████████| 599105/599105 [09:18<00:00, 1072.82it/s]

In [10]:
import pandas as pd
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, Y_train_preds_filtered = filter_unlabeled_dataframe(
    df_train, Y_train_preds, L_train
)
df_train_filtered["rating"] = Y_train_preds_filtered
combined_df_train = pd.concat([df_train_filtered, df_dev], axis=0)

/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


### Rating Prediction Model
We write a Keras model for predicting ratings given a user's book list and a book (which is being rated).
The model represents the list of books the user interacted with, `books_idxs`, by learning an embedding for each idx, and averaging the embeddings in `book_idxs`.
It learns another embedding for the `book_idx`, the book to be rated.
Then it concatenates the two embeddings and uses an [MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron) to compute the probability of the `rating` being 1.
This type of model is common in large-scale recommender systems, for example, the [YouTube recommender system](https://ai.google/research/pubs/pub45530).

In [11]:
import numpy as np
import tensorflow as tf
from utils import precision_batch, recall_batch, f1_batch

n_books = len(df_books)


# Keras model to predict rating given book_idxs and book_idx.
def get_model(embed_dim=64, hidden_layer_sizes=[32]):
    # Compute embedding for book_idxs.
    len_book_idxs = tf.keras.layers.Input([])
    book_idxs = tf.keras.layers.Input([None])
    book_idxs_emb = tf.keras.layers.Embedding(n_books, embed_dim)(book_idxs)
    book_idxs_emb = tf.math.divide(
        tf.keras.backend.sum(book_idxs_emb, axis=1), tf.expand_dims(len_book_idxs, 1)
    )
    # Compute embedding for book_idx.
    book_idx = tf.keras.layers.Input([])
    book_idx_emb = tf.keras.layers.Embedding(n_books, embed_dim)(book_idx)
    input_layer = tf.keras.layers.concatenate([book_idxs_emb, book_idx_emb], 1)
    # Build Multi Layer Perceptron on input layer.
    cur_layer = input_layer
    for size in hidden_layer_sizes:
        tf.keras.layers.Dense(size, activation=tf.nn.relu)(cur_layer)
    output_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(cur_layer)
    # Create and compile keras model.
    model = tf.keras.Model(
        inputs=[len_book_idxs, book_idxs, book_idx], outputs=[output_layer]
    )
    model.compile(
        "Adagrad", "binary_crossentropy", metrics=["accuracy", f1_batch, precision_batch, recall_batch]
    )
    return model

We use triples of (`book_idxs`, `book_idx`, `rating`) from our dataframes as training examples. In addition, we want to train the model to recognize when a user will not read a book. To create examples for that, we randomly sample a `book_id` not in `book_idxs` and use that with a `rating` of 0 as a _random negative_ example. We create one such _random negative_ example for every positive (`rating` 1) example in our dataframe so that positive and negative examples are roughly balanced.

In [12]:
# Generator to turn dataframe into examples.
def get_examples_generator(df):
    def generator():
        for book_idxs, book_idx, rating in zip(df.book_idxs, df.book_idx, df.rating):
            # Remove book_idx from book_idxs so the model can't just look it up.
            book_idxs = tuple(filter(lambda x: x != book_idx, book_idxs))
            yield {
                "len_book_idxs": len(book_idxs),
                "book_idxs": book_idxs,
                "book_idx": book_idx,
                "label": rating,
            }
            if rating == 1:
                # Generate a random negative book_id not in book_idxs.
                random_negative = np.random.randint(0, n_books)
                while random_negative in book_idxs:
                    random_negative = np.random.randint(0, n_books)
                yield {
                    "len_book_idxs": len(book_idxs),
                    "book_idxs": book_idxs,
                    "book_idx": random_negative,
                    "label": 0,
                }

    return generator


def get_data_tensors(df):
    # Use generator to get examples each epoch, along with shuffling and batching.
    padded_shapes = {
        "len_book_idxs": [],
        "book_idxs": [None],
        "book_idx": [],
        "label": [],
    }
    dataset = (
        tf.data.Dataset.from_generator(
            get_examples_generator(df), {k: tf.int64 for k in padded_shapes}
        )
        .shuffle(123)
        .repeat(None)
        .padded_batch(batch_size=256, padded_shapes=padded_shapes)
    )
    tensor_dict = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    return (
        tensor_dict["len_book_idxs"],
        tensor_dict["book_idxs"],
        tensor_dict["book_idx"],
        tensor_dict["label"],
    )

We now train the model on our combined training data (data labeled by LFs plus dev data).


In [13]:
model = get_model()

train_data_tensors = get_data_tensors(combined_df_train)
val_data_tensors = get_data_tensors(df_val)
model.fit(
    train_data_tensors[:-1],
    train_data_tensors[-1],
    steps_per_epoch=300,
    validation_data=(val_data_tensors[:-1], val_data_tensors[-1]),
    validation_steps=40,
    epochs=30,
    verbose=1,
)

W0811 07:43:47.171001 140048767420224 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W0811 07:43:47.250380 140048767420224 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


W0811 07:43:47.366583 140048767420224 deprecation.py:323] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0811 07:43:47.388775 140048767420224 deprecation.py:323] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


W0811 07:43:47.576594 140048767420224 deprecation.py:506] From /home/ubuntu/snorkel-tutorials/.tox/recsys/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/adagrad.py:105: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

293/300 [============================>.] - ETA: 0s - loss: 0.5941 - acc: 0.7090 - f1_batch: 0.4629 - precision_batch: 0.8172 - recall_batch: 0.3285

295/300 [============================>.] - ETA: 0s - loss: 0.5944 - acc: 0.7087 - f1_batch: 0.4621 - precision_batch: 0.8159 - recall_batch: 0.3279

297/300 [============================>.] - ETA: 0s - loss: 0.5947 - acc: 0.7083 - f1_batch: 0.4618 - precision_batch: 0.8149 - recall_batch: 0.3278

299/300 [============================>.] - ETA: 0s - loss: 0.5949 - acc: 0.7082 - f1_batch: 0.4621 - precision_batch: 0.8141 - recall_batch: 0.3282

300/300 [==============================] - 12s 41ms/step - loss: 0.5951 - acc: 0.7080 - f1_batch: 0.4616 - precision_batch: 0.8134 - recall_batch: 0.3279 - val_loss: 0.6484 - val_acc: 0.6553 - val_f1_batch: 0.3994 - val_precision_batch: 0.6073 - val_recall_batch: 0.3065


Finally, we evaluate the model's predicted ratings on our test data.


In [14]:
test_data_tensors = get_data_tensors(df_test)
_ = model.evaluate(test_data_tensors[:-1], test_data_tensors[-1], steps=30)

 1/30 [>.............................] - ETA: 3s - loss: 0.6580 - acc: 0.6367 - f1_batch: 0.3111 - precision_batch: 0.4773 - recall_batch: 0.2308

 3/30 [==>...........................] - ETA: 1s - loss: 0.6681 - acc: 0.6367 - f1_batch: 0.3763 - precision_batch: 0.6237 - recall_batch: 0.2698

 5/30 [====>.........................] - ETA: 1s - loss: 0.6526 - acc: 0.6539 - f1_batch: 0.3661 - precision_batch: 0.6363 - recall_batch: 0.2578

 7/30 [======>.......................] - ETA: 0s - loss: 0.6452 - acc: 0.6613 - f1_batch: 0.3793 - precision_batch: 0.6730 - recall_batch: 0.2650

 9/30 [========>.....................] - ETA: 0s - loss: 0.6473 - acc: 0.6597 - f1_batch: 0.3908 - precision_batch: 0.6708 - recall_batch: 0.2770

...

22/30 [=====================>........] - ETA: 0s - loss: 0.6515 - acc: 0.6605 - f1_batch: 0.4108 - precision_batch: 0.6698 - recall_batch: 0.3004

24/30 [=======================>......] - ETA: 0s - loss: 0.6516 - acc: 0.6613 - f1_batch: 0.4158 - precision_batch: 0.6698 - recall_batch: 0.3058

26/30 [=========================>....] - ETA: 0s - loss: 0.6506 - acc: 0.6629 - f1_batch: 0.4200 - precision_batch: 0.6818 - recall_batch: 0.3077

28/30 [===========================>..] - ETA: 0s - loss: 0.6491 - acc: 0.6648 - f1_batch: 0.4231 - precision_batch: 0.6755 - recall_batch: 0.3131

30/30 [==============================] - 1s 31ms/step - loss: 0.6504 - acc: 0.6630 - f1_batch: 0.4173 - precision_batch: 0.6661 - recall_batch: 0.3086


Our model has generalized quite well to our test set!
Note that we should additionally measure ranking metrics, like precision@10, before deploying to production.

## Summary

In this tutorial, we showed one way to use Snorkel for recommendations.
We used book metadata and review text to create LFs that estimate user ratings.
We used Snorkel's `LabelModel` to combine the outputs of those LFs.
Finally, we trained a model to predict whether a user will read and like a given book (and therefore what books should be recommended to the user) based only on what books the user has interacted with in the past.

Here we demonstrated one way to use Snorkel for training a recommender system.
Note, however, that this approach could easily be adapted to take advantage of additional information as it is available (e.g., user profile data, denser user ratings, and so on.)